# Counting results

In [ ]:
import pandas as pd
from matplotlib_venn import venn3
from matplotlib import pyplot as plt

## 1. Journals

Taxonomic journals:
- with title
- found on wikidata (each method)
- found on OpenAlex
- total
- still publishing in the last 10 years
- dissolved (confirmed)
- with ISSN-L
- with IPNI publication ID
- with ZooBank publication ID
- with OpenAlex ID

In [ ]:
journals = pd.read_csv("../data/processed/journals_deduplicated.csv")
journals.head()

In [ ]:
print("Number of journals: " + str(len(journals)))
print("Number of journals with known title: "+\
      str(len([x for x in journals["title"] if x == x])))

In [ ]:
table = pd.DataFrame({"name":["Taxonomic journals with title",
                             "Journals with Wikidata ID",
                             "Journals with ISSN-L",
                             "Journals with IPNI Publication ID in Wikidata",
                             "Journals with ZooBank Publication ID in Wikidata",
                             "Journals with OpenAlex ID",
                            # union of OpenAlex ID and IPNI etc
                             "Journals with useful taxonomic journals"],
                      "value":[0,0,0,0,0,0,0,0,0]})

#table = table.set_index("name")
table

In [ ]:
table.loc[0,"value"] = str(len([x for x in journals["title"] if x == x]))

In [ ]:
table

In [ ]:
for i, column in enumerate(journals.columns[1:6]):
    print("Number of journals with " + column + ": "+\
          str(len([x for x in journals[column] if x == x])))
    table.loc[i+1, "value"] = str(len([x for x in journals[column] if x == x]))

In [ ]:
table

In [ ]:
print("Journals that were dissolved (confirmed): ")
journals["dissolved"].value_counts()

In [ ]:
table.loc[6,"value"] = str(len([x for x in journals["dissolved"] if x == True]))
table.loc[7,"value"] = str(len([x for x in journals["dissolved"] if x == False]))

In [ ]:
table # ctrl + click to select cells to copy

In [ ]:
articles = pd.read_pickle("../data/interim/filtered_articles.pkl")
articles.head()

In [ ]:
print("Number of journals that contained relevant (i.e. taxonomic) articles: " + \
      str(len(set(articles["source_id"]))))

In [ ]:
# Venn diagram
ipnizoo = set(journals[journals["source"]=="IPNI or ZooBank ID"]["title"])
openalex = set(journals[journals["source"]=="OpenAlex taxonomy concept"]["title"])
wikisubjects = set(journals[journals["source"]=="Wikidata taxonomic subject"]["title"])

venn3([ipnizoo, openalex, wikisubjects], ("IPNI/ZooBank ID", "OpenAlex", "Wikidata subjects"))
plt.title("Unique journals per source", fontsize=15)

## 2. Articles

- total number
- number with at least one European author

In [ ]:
print("Number of articles related to taxonomy: " + str(len(set(articles["id"]))))

In [ ]:
eu_articles = pd.read_pickle("../data/interim/filtered_articles.pkl")

In [ ]:
print("Number of articles related to taxonomy with at least one European author: " +\
      str(len(set(eu_articles["id"]))))

## 3. Authors

In [ ]:
eu_authors = pd.read_csv("../data/processed/country_taxonomic_authors_no_duplicates.tsv", sep="\t")

In [ ]:
print("Number of European authors before disambiguation: "+\
      str(len(eu_authors)))

In [ ]:
eu_disamb_authors = pd.read_pickle("../data/processed/authors_disambiguated_truncated.pkl")

In [ ]:
print("Number of European authors after disambiguation: "+\
      str(len(eu_disamb_authors)))

In [ ]:
print("Number of European institutions publishing taxonomic articles: "+\
      str(len(set(eu_disamb_authors["inst_id"]))))

In [ ]:
len(set(eu_authors[eu_authors["inst_country_code"]=="BE"]["inst_display_name"]))

In [ ]:
eu_authors.columns

## 4. Taxonomy

Total number of families within Plantae:
Total number of families within Animalia:
Total number of families within Fungi:
Total number of families within Protists:
…. See similar figure here: https://data-blog.gbif.org/post/gbif-backbone-taxonomy/

In [ ]:
# supply and demand order save?
authors_tree = pd.read_pickle("../data/processed/authors_disambiguated_truncated.pkl")

In [ ]:
authors2 = pd.read_pickle("../data/interim/country_taxonomic_authors_no_duplicates.pkl")

In [ ]:
authors2.head()

In [ ]:
authors_tree.head()

In [ ]:
def get_number_families(kingdom_name):
    kingdom = []

    for author in authors_tree.itertuples():
        if kingdom_name in author.kingdoms_subjects:
            kingdom.append(author)
            
    kingdom_df = pd.DataFrame(kingdom)
    
    families = []
    
    for author in kingdom_df.itertuples():
        for family in author.families_subjects:
            families.append(family)
            
    return len(set(families))

In [ ]:
print("Number of plant families in data: " + str(get_number_families("Plantae")))
print("Number of animal families in data: " + str(get_number_families("Animalia")))
print("Number of fungal families in data: " + str(get_number_families("Fungi")))
# no protists in data

## 5. Most prolific authors

In [ ]:
authors = pd.read_pickle("../data/interim/all_authors_of_european_taxonomic_articles.pkl")

In [ ]:
top_authors = authors["author_id"].value_counts()[0:10]

In [ ]:
authors.columns

In [ ]:
sum(list(authors["author_id"].value_counts()))/len(list(authors["author_id"].value_counts()))

In [ ]:
for author_id in top_authors.index:
    print(authors[authors["author_id"]==author_id].iloc[0]["author_display_name"] + \
          " has written " + str(top_authors.loc[author_id]) + " articles")

In [ ]:
plt.xlim(1,30)
plt.hist(authors["author_id"].value_counts(), bins=150)

## 6. ORCID and Open Access percentages

In [ ]:
# total percentage of authors with ORCID
eu_disamb_authors["author_orcid"]

In [ ]:
orcid = 0
no_orcid = 0

for x in eu_disamb_authors["author_orcid"]:
    if isinstance(x, str):
        if x.split(".org/")[0] == "https://orcid":
            orcid += 1
        """ # checking if there are ORCIDs in lists didn't change the results
        elif isinstance(x, tuple) or isinstance(x, set) or isinstance(x, list):
            if isinstance(x[0], str):
                if x[0].split(".org/")[0] == "https://orcid":
                    orcid += 1
                else:
                    no_oricd += 1
            else:
                no_orcid += 1
        """
        else:
            no_orcid += 1
    else:
        no_orcid += 1
        
print("Percentage of authors with an ORCID: " + str(orcid/(no_orcid+orcid)*100))

In [ ]:
articles = pd.read_pickle("../data/processed/european_taxonomic_articles_with_subjects.pkl")

In [ ]:
print("Percentage of European articles that are Open Access: " + \
      str(articles["oa_is_oa"].value_counts()[0]/(articles["oa_is_oa"].value_counts()[0]+articles["oa_is_oa"].value_counts()[1])*100))

In [ ]:
print("Percentage of European articles that are Open Access: " + \
      str(articles["oa_is_oa"].value_counts()[0]/(articles["oa_is_oa"].value_counts()[0]+articles["oa_is_oa"].value_counts()[1])*100))

In [ ]:
articles.columns

In [ ]:
def species_to_tree(df, backbone):
    # make dictionary of genus, family, order, class, phylum, kingdom for every species
    seen_species = {}

    for species in backbone.itertuples():
        if species.canonicalName not in seen_species:
            seen_species[species.canonicalName] = list(species)[2:]
    
    # associate the full tree with a certain author or article
    genera, families, orders, classes, phyla, kingdoms, lineages = [], [], [], [], [], [], []

    for row in df.itertuples():
        genus, family, order, tclass, phylum, kingdom, lineage = [], [], [], [], [], [], []
                            # python won't allow class as a variable name

        for species in row.species_subject:
            if species in seen_species:
                genus.append(seen_species[species][-1])
                family.append(seen_species[species][-2])
                order.append(seen_species[species][-3])
                tclass.append(seen_species[species][-4])
                phylum.append(seen_species[species][-5])
                kingdom.append(seen_species[species][-6])
                
                lineage.append(seen_species[species][-6:])

        genera.append(set(genus))
        families.append(set(family))
        orders.append(set(order))
        classes.append(set(tclass))
        phyla.append(set(phylum))
        kingdoms.append(set(kingdom))
        lineages.append(lineage)

    df["genera_subjects"] = genera
    df["families_subjects"] = families
    df["orders_subjects"] = orders
    df["classes_subjects"] = classes
    df["phyla_subjects"] = phyla
    df["kingdoms_subjects"] = kingdoms
    df["lineages_subjects"] = lineages
    
    return df

In [ ]:
def preprocess_backbone(path="../data/external/backbone/Taxon.tsv", no_blanks=False):
    # GBIF taxonomic bakcbone
    backbone = pd.read_csv(path, sep="\t", on_bad_lines='skip')

    # only Eukarya
    backbone = backbone[(backbone["kingdom"]=="Animalia") | 
                        (backbone["kingdom"]=="Plantae") |
                        (backbone["kingdom"]=="Fungi")]
    # include non-accepted species (synonyms etc), but not blank canonical names 
    backbone = backbone[np.logical_not(backbone["canonicalName"].isnull())].reset_index(drop=True)
    
    if no_blanks:
        backbone = backbone[np.logical_not(backbone["genus"].isnull())].reset_index(drop=True)
        backbone = backbone[np.logical_not(backbone["family"].isnull())].reset_index(drop=True)
        backbone = backbone[np.logical_not(backbone["order"].isnull())].reset_index(drop=True)
        backbone = backbone[np.logical_not(backbone["class"].isnull())].reset_index(drop=True)
        backbone = backbone[np.logical_not(backbone["phylum"].isnull())].reset_index(drop=True)
        backbone = backbone[np.logical_not(backbone["kingdom"].isnull())].reset_index(drop=True)

    backbone = backbone.drop_duplicates(ignore_index=True)
    return backbone

In [ ]:
import numpy as np

In [ ]:
backbone = preprocess_backbone()

In [ ]:
articles2 = species_to_tree(articles, backbone)

In [ ]:
plants_open = 0
plants_closed = 0

animals_open = 0
animals_closed = 0

fungi_open = 0
fungi_closed = 0

for row in articles2.itertuples():
    if "Plantae" in row.kingdoms_subjects:
        if row.oa_is_oa:
            plants_open += 1
        else:
            plants_closed += 1
            
    if "Animalia" in row.kingdoms_subjects:
        if row.oa_is_oa:
            animals_open += 1
        else:
            animals_closed += 1
            
    if "Fungi" in row.kingdoms_subjects:
        if row.oa_is_oa:
            fungi_open += 1
        else:
            fungi_closed += 1

In [ ]:
print("Percentage of articles about plants that are open access: " + \
      str(plants_open/(plants_open+plants_closed)*100))
print("Percentage of articles about animals that are open access: " + \
      str(animals_open/(animals_open+animals_closed)*100))
print("Percentage of articles about fungi that are open access: " + \
      str(fungi_open/(fungi_open+fungi_closed)*100))